# CNN Network : Create
(CNN Network를 생성해 준다. ID가 같은 CNN Network가 있으면 재 생성 하지 않는다.)<br>

nn_id : Neural Network ID<br>
biz_cate : Business 대분류<br>
biz_sub_cate : Business 소분류<br>
nn_title : Neural Network Title<br>
nn_desc : Neural Network Description<br>
nn_wf_ver_info : Neural Network Workflow Info<br>
use_flag : 사용여부<br>
dir : 문제유형<br>
config : Custom 여부<br>
network_type : network<br>
node_sub_menu : Trainning & Eval Data menu<br>

In [1]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00004"
biz_cate = "ERP"
biz_sub_cate = "MRO"
nn_title = "MRO Image Classification"
nn_desc = "MRO Image Classification"
nn_wf_ver_info = "MRO Image Classification"
use_flag = "Y"
dirstr = "purpose?"
config = "N"
network_type = "cnn"
node_sub_menu = "data_image"

def spaceprint(val, cnt):
    leng = len(str(val))
    cnt = cnt - leng
    restr = ""
    for i in range(cnt):
        restr += " "
    restr = restr+str(val)
    return restr

#insert nn_info
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+ '/',
                     json={
                         "nn_id": nn_id,
                         "biz_cate": biz_cate,
                         "biz_sub_cate": biz_sub_cate,
                         "nn_title" : nn_title,
                         "nn_desc": nn_desc,
                         "use_flag" : use_flag,
                         "dir": dirstr,
                         "config": config
                     })
data = json.loads(resp.json())
print("insert nn_info evaluation result : {0}".format(data))

insert nn_info evaluation result : nn00004


# CNN Network WorkFlow : Create
(CNN Network WorkFlow를 생성해 준다. 실행할 때마다 신규 버전을 새로 생성 한다.)<br>

nn_def_list_info_nn_id : 값을 넘기지 않아도 됨.<br>
nn_wf_ver_info : Work Flow Version Description<br>
condition :<br>
active_flag : 사용여부<br>


In [12]:
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/',
                     json={
                         "nn_def_list_info_nn_id": "",
                         "nn_wf_ver_info": nn_wf_ver_info,
                         "condition": "1",
                         "active_flag": "N"
                     })
data = json.loads(resp.json())
print("insert nn_info Work Flow Create")

insert nn_info Work Flow Create


# CNN Network WorkFlow : Get Active Version 
(여러개의 CNN Network WorkFlow중 Active한 Version을 가져온다.)<br>

In [2]:
# get workflow version info
resp = requests.get('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/')
data = json.loads(resp.json())

# get Active workflow version
wf_ver_id = 0
data = sorted(data, key=lambda k: k['pk'])
for config in data:
    print("Version="+str(config["pk"])+" active="+str(config["fields"]["active_flag"]))
    if config["fields"]["active_flag"] == "Y":
        wf_ver_id = config["pk"]

wf_ver_id = str(wf_ver_id)
print("Active Version Workflow ID="+wf_ver_id)

# get Max workflow version
wf_ver_id = 0
for i in data:
    if i["pk"] > wf_ver_id:
        wf_ver_id = i["pk"]

wf_ver_id = str(wf_ver_id)

Version=1 active=N
Version=2 active=N
Version=3 active=N
Version=4 active=N
Version=5 active=N
Version=6 active=N
Version=7 active=N
Version=8 active=N
Version=9 active=N
Version=10 active=N
Version=11 active=N
Version=12 active=N
Version=13 active=N
Version=14 active=N
Version=15 active=Y
Active Version Workflow ID=15


# CNN Network WorkFlow : Set Active Version
(여러개의 CNN Network WorkFlow중 특정 Version을 Active 시킨다. Active한 Version은 한 개만 존재 할 수 있다.)<br>

wf_ver_id : active version

In [3]:
# update workflow version info
resp = requests.put('http://' + url + '/api/v1/type/common/target/nninfo/nnid/'+nn_id+'/version/',
                     json={
                         "nn_wf_ver_id": wf_ver_id,
                         "nn_def_list_info_nn_id": "",
                         "nn_wf_ver_info": nn_wf_ver_info,
                         "condition": "1",
                         "active_flag": "Y"
                     })
data = json.loads(resp.json())
print("Update active workflow version info evaluation result : {0}".format(data))

Update active workflow version info evaluation result : 10


# CNN Network WorkFlow Node : Create
(CNN Network WorkFlow Node를 생성해 준다. 기존 Node가 있으면 재 생성 하지 않는다.)<br>

In [16]:
# insert workflow version node info
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/'+nn_id+'/wfver/'+wf_ver_id+'/',
                     json={
                         "type": network_type
                     })
data = json.loads(resp.json())
print("insert workflow version node info evaluation result : {0}".format(data))

insert workflow version node info evaluation result : cnn


# CNN Network WorkFlow Node : Network Config Setup
(CNN Network WorkFlow Node의 Network Config를 Setup 해준다.)<br>
learnrate : Tensorflow의 Optimizer 학습 속도
traincnt : 학습 카운트
batch_size : 이미지 개수
num_classes : 학습 Category 개수
predictcnt : 예측 후 리턴 받는 Category 개수
layeroutputs : 각 layer output size

In [3]:
node = "netconf_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                         "param":{"epoch":2
                                  ,"traincnt": 1,
                                     "batch_size":10000,
                                     "predictcnt": 10
                         },
                         "config": {"num_classes":10,
                                    "learnrate": 0.001,
                                     "layeroutputs":32,
                                     "type":"category"
                                     }
                         ,"layer1": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":2
                                    }
                         ,"layer2": {"type": "cnn",
                                     "active": "relu",
                                     "cnnfilter": [3, 3],
                                     "cnnstride": [1, 1],
                                     "maxpoolmatrix": [2, 2],
                                     "maxpoolstride": [2, 2],
                                     "padding": "SAME",
                                     "droprate": "0.8",
                                     "layercnt":1
                                    }
                          ,"out": {"active": "softmax",
                                   "node_out": 625,
                                   "padding": "SAME"
                                }
                           ,"labels":[]
                        })
netconf = json.loads(resp.json())
print("insert workflow node conf info evaluation result : {0}".format(netconf))

insert workflow node conf info evaluation result : {'param': {'epoch': 2, 'traincnt': 1, 'predictcnt': 10, 'batch_size': 10000}, 'config': {'layeroutputs': 32, 'num_classes': 10, 'learnrate': 0.001, 'type': 'category'}, 'layer1': {'type': 'cnn', 'padding': 'SAME', 'maxpoolmatrix': [2, 2], 'droprate': '0.8', 'cnnfilter': [3, 3], 'layercnt': 2, 'active': 'relu', 'maxpoolstride': [2, 2], 'cnnstride': [1, 1]}, 'labels': ['car', 'airplane', 'bolt', 'motor', 'glove'], 'layer2': {'type': 'cnn', 'padding': 'SAME', 'maxpoolmatrix': [2, 2], 'droprate': '0.8', 'cnnfilter': [3, 3], 'layercnt': 1, 'active': 'relu', 'maxpoolstride': [2, 2], 'cnnstride': [1, 1]}, 'out': {'padding': 'SAME', 'node_out': 625, 'active': 'softmax'}}


# CNN Network WorkFlow Node :  Data Config Setup
(CNN Network WorkFlow Node의 Data Config를 Setup 해준다.)<br>
x_size : 이미지 가로 크기<br>
y_size : 이미지 세로 크기<br>
channel : 3 컬러 1 흑백<br>
labels : 입력 하지 않는다.<br>

In [5]:
node = "datasrc"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                         

                     })
dataconf = json.loads(resp.json())
print("insert workflow node conf info evaluation result : {0}".format(dataconf))

insert workflow node conf info evaluation result : {'source_path': '/hoya_src_root/nn00004/10/datasrc', 'store_path': '/hoya_str_root/nn00004/10/datasrc', 'preprocess': {'x_size': 32, 'y_size': 32, 'channel': 3}, 'labels': ['dog', 'cat', 'ca    t', 'motor', 'car', 'glove', 'airplane', 'bolt']}


# CNN Network WorkFlow Node :  Eval Config Setup

In [6]:
node = "eval_node"
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/cnn/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',json={})
evalconf = json.loads(resp.json())
print("insert workflow node conf info evaluation result : {0}".format(evalconf))

insert workflow node conf info evaluation result : {'modelname': 'model_nn00004_10', 'modelpath': '/hoya_model_root/nn00004/10/eval_node'}


# CNN Network WorkFlow Node :  Eval Data Config Setup
(CNN Network WorkFlow Node의 Data Config를 Setup 해준다.)<br>
x_size : 이미지 가로 크기<br>
y_size : 이미지 세로 크기<br>
channel : 3 컬러 1 흑백<br>
labels : 입력 하지 않는다.<br>

In [7]:
node = 'evaldata'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/imgdata/src/local/form/file/prg/source/nnid/'+nn_id+'/ver/'+wf_ver_id+'/node/'+node+'/',
                     json={
                            "preprocess": {"x_size": 32,
                                        "y_size": 32,
                                        "channel":3}
                     })
edataconf = json.loads(resp.json())
print("insert workflow node conf info evaluation result : {0}".format(edataconf))

insert workflow node conf info evaluation result : {'source_path': '/hoya_src_root/nn00004/10/evaldata', 'store_path': '/hoya_str_root/nn00004/10/evaldata', 'preprocess': {'x_size': 32, 'y_size': 32, 'channel': 3}, 'labels': ['motor', 'car', 'glove', 'airplane', 'bolt']}


# Trainning & Eval Data Transfer
(CNN Network Train을 할 수 있게 Data를 특정 장소에 전송해 준다.)<br>

In [35]:
# get workflow node list info (train, eval)
resp = requests.get('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+node_sub_menu+'/')
datalist = json.loads(resp.json())

print("get file node list")
for node in datalist:
    print(datalist[node])
print("")

# Train Folder에 Data를 위치 시킬지 Eval Folder에 Data를 위치 시킬지 결정후 전송해야 한다.
print("create file node list")
for node in datalist:
    typepath = datalist[node]["nn_wf_node_name"]

    files = {'file1': open('/home/dev/hoyai/demo/data/cat_vs_dog.zip','rb'),
             'file2': open('/home/dev/hoyai/demo/data/sample_cnn_img.zip','rb')}

    resp = requests.post('http://' + url + '/api/v1/type/wf/state/data/detail/upload/file/nnid/'+nn_id+'/ver/'+wf_ver_id+'/dir/'+typepath+'/',
                         files=files)
    data = json.loads(resp.json())
    print(data)

get file node list
{'nn_wf_node_name': 'datasrc', 'nn_wf_node_id': 'nn00004_41_datasrc', 'nn_wf_node_desc': 'Train Data Node'}
{'nn_wf_node_name': 'evaldata', 'nn_wf_node_id': 'nn00004_41_evaldata', 'nn_wf_node_desc': 'Evaluation Data Node'}

create file node list
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/41/datasrc/sample_cnn_img.zip2)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/41/datasrc/sample_cnn_img.zip2)'}}
{'File1': {'File': 'File Create.(/hoya_src_root/nn00004/41/evaldata/sample_cnn_img.zip2)'}, 'File0': {'File': 'File Create.(/hoya_src_root/nn00004/41/evaldata/sample_cnn_img.zip2)'}}


# CNN Network Training
(CNN Network Training을 실행한다.)<br>

In [8]:
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+wf_ver_id+'/')
data = json.loads(resp.json())

for train in data:
    if train != None and train != "" and train != {} and train != "status" and train != "result":
        try:
            for tr in train["TrainResult"]:
                print(tr)
        except:
            maxcnt = 0
            line = ""
            for label in train["labels"]:
                if maxcnt<len(label):
                    maxcnt = len(label)

            for i in range(len(train["labels"])):
                for j in range(maxcnt+4):
                    line += "="

            label_sub = []
            for label in train["labels"]:
                label = spaceprint(label,maxcnt)
                label_sub.append(label)

            space = ""
            for s in range(maxcnt):
                space +=" "

            print(space, label_sub)
            print(space, line)
            for i in range(len(train["labels"])):
                predict_sub = []
                for j in range(len(train["predicts"][i])):
                    pred = spaceprint(train["predicts"][i][j],maxcnt)

                    predict_sub.append(pred)

                print(spaceprint(train["labels"][i],maxcnt), predict_sub)

['Trainning .................................................']
['Global Step:     1315, Training Batch Accuracy:  100.0%']

['Trainning .................................................']
['Global Step:     1316, Training Batch Accuracy:  99.19%']

         ['     dog', '     cat', ' ca    t', '   motor', '     car', '   glove', 'airplane', '    bolt']
     dog ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
     cat ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
 ca    t ['       0', '       0', '       0', '       0', '       0', '       0', '       0', '       0']
   motor ['       0', '       0', '       0', '     144', '       0', '       0', '       1', '       5']
     car ['       0', '       0', '       0', '       2', '     142', '       1', '       2', '       3']
   glove ['       0', '       0', '       0', '       1', '       0', '     144', '       0', '       2']
airplane

# CNN Network Predict
(CNN Network Predict를 실행한다.)<br>

In [9]:
files = {
         'files000001':  open('/home/dev/hoyai/demo/data/airplane/1air.jpg','rb')
        ,'files000002':  open('/home/dev/hoyai/demo/data/airplane/2air.jpg','rb')
        ,'files000003':  open('/home/dev/hoyai/demo/data/bolt/1bolt.jpg','rb')
        ,'files000004':  open('/home/dev/hoyai/demo/data/bolt/2bolt.jpg','rb')
        ,'files000005':  open('/home/dev/hoyai/demo/data/car/1car.jpg','rb')
        ,'files000006':  open('/home/dev/hoyai/demo/data/car/2car.jpg','rb')
        ,'files000007':  open('/home/dev/hoyai/demo/data/glove/1glove.jpg','rb')
        ,'files000008':  open('/home/dev/hoyai/demo/data/glove/2glove.jpg','rb')
        ,'files000009':  open('/home/dev/hoyai/demo/data/motor/1motor.jpg','rb')
        ,'files000010':  open('/home/dev/hoyai/demo/data/motor/2motor.jpg','rb')
        }
restURL = 'http://' + url + '/api/v1/type/service/state/predict/type/'+network_type+'/nnid/'+nn_id+'/ver/'+wf_ver_id+'/'

resp = requests.post(restURL,
                     files=files
                     )
data = json.loads(resp.json())

for train in data:
    print("FileName = "+train)
    print(data[train]['key'])
    print(data[train]['val'])
    print('')

FileName = 1glove.jpg
['glove', 'motor', 'car', 'bolt', 'cat', 'dog', 'airplane', 'ca    t']
[19.07092, 5.76438, 5.46667, 0.29189, -4.39018, -4.43267, -6.54681, -9.49485]

FileName = 1motor.jpg
['motor', 'bolt', 'car', 'airplane', 'glove', 'cat', 'ca    t', 'dog']
[10.16726, 3.84742, 1.90861, 1.74771, 1.39918, -3.10327, -6.24498, -6.50998]

FileName = 2motor.jpg
['motor', 'car', 'bolt', 'cat', 'airplane', 'glove', 'ca    t', 'dog']
[3.47767, 1.12429, 1.11521, -0.47072, -1.10687, -1.50571, -1.74819, -2.46043]

FileName = 1car.jpg
['car', 'cat', 'bolt', 'motor', 'airplane', 'glove', 'dog', 'ca    t']
[26.53919, 9.05163, 3.93843, 0.26702, -0.09171, -1.74729, -6.97068, -8.76143]

FileName = 2air.jpg
['airplane', 'bolt', 'car', 'motor', 'glove', 'cat', 'ca    t', 'dog']
[9.78897, 4.0355, 0.27341, -1.41575, -2.0793, -3.11359, -3.3978, -7.41948]

FileName = 1bolt.jpg
['bolt', 'car', 'glove', 'motor', 'airplane', 'cat', 'ca    t', 'dog']
[20.41062, 4.08477, 2.86333, 2.74491, 2.29809, -8.4897, 